# Building the decision tree classifier

In [1]:
import numpy as np
import pandas as pd

# Enter You Name Here
myname = "Tamal-Mondal-"

# Writing results to a file (DO NOT CHANGE)
f = open(myname + "result.txt", "w")

# Structure of a single node in the decision tree
class TreeNode():
    def __init__(self, left_child=None, right_child=None, feature=None, threshold_value = None, class_label=None):
        
        # for internal or decision node
        self.left_child = left_child
        self.right_child = right_child
        self.feature = feature
        self.threshold_value = threshold_value
        
        # This is only applicable to leaf node to capture the class label
        self.class_label = class_label

class DecisionTree():
    def __init__(self, information_gain_strategy, minimum_datapoints_to_split, maximum_height):
        self.root = None
        self.information_gain_strategy = information_gain_strategy

        # These are the parameters to do pre-pruning to avoid overfitting
        self.minimum_datapoints_to_split = minimum_datapoints_to_split
        self.maximum_height = maximum_height

    # Method that recursively gets called and builds the decision tree
    def generate_decision_tree(self, datapoints, current_height):
        number_of_data_points = np.shape(datapoints)[0]

        # Only try to split if the tree has not reached maximum height and there are sufficient no of data points
        if number_of_data_points >= self.minimum_datapoints_to_split and current_height < self.maximum_height:

            # Find the best split that maximizes the information gain
            spliting_details = self.find_best_split_with_threshold(datapoints)

            # Split only if there is valid information gain
            if spliting_details["information_gain"] > 0:
                right_tree = self.generate_decision_tree(spliting_details["datapoints_to_right"], current_height + 1)
                left_tree = self.generate_decision_tree(spliting_details["datapoints_to_left"], current_height + 1)
                return TreeNode(left_tree, right_tree, spliting_details["feature"], spliting_details["threshold_value"])
        
        # Otherwise create a leaf node by majority count
        return TreeNode(class_label = find_class_label_by_majority_vote(datapoints[:,-1]))

    # Method to try all features and their thresolds to find 
    # the best way to split depending on information gain
    def find_best_split_with_threshold(self, datapoints):

        # It stores the details related to best split
        spliting_details = {"information_gain" : float(-np.inf)}
        
        # Check all unique feature values as thresolds for all the features
        for index in range(np.shape(datapoints)[1] - 1):
            for threshold_value in np.unique(datapoints[:, index]):
                
                # Split the dataset by thresold for a given feature
                datapoints_to_left = np.array([datapoint for datapoint in datapoints if datapoint[index] <= threshold_value])
                datapoints_to_right = np.array([datapoint for datapoint in datapoints if datapoint[index] > threshold_value])

                # Modify split details if a better information gain is found
                if len(datapoints_to_left) >= 1 and len(datapoints_to_right) >= 1:
                    information_gain = self.calculate_information_gain(datapoints[:, -1], datapoints_to_left[:, -1], datapoints_to_right[:, -1])
                    if information_gain > spliting_details["information_gain"]:
                        spliting_details["information_gain"]  = information_gain
                        spliting_details["datapoints_to_left"] = datapoints_to_left
                        spliting_details["datapoints_to_right"] = datapoints_to_right
                        spliting_details["feature"] = index
                        spliting_details["threshold_value"] = threshold_value
        return spliting_details

    # Method to compute information gain by "entropy" or "gini index"
    def calculate_information_gain(self, y_parent, y_left_tree, y_right_tree):
        left_child_weight = len(y_left_tree) / len(y_parent)
        right_child_weight = len(y_right_tree) / len(y_parent)
        information_gain = float(-np.inf)
        if self.information_gain_strategy == "entropy":
            information_gain = compute_entropy(y_parent) - (left_child_weight * compute_entropy(y_left_tree) + right_child_weight * compute_entropy(y_right_tree))
        elif self.information_gain_strategy == "gini":
            information_gain = compute_gini_index(y_parent) - (left_child_weight * compute_gini_index(y_left_tree) + right_child_weight * compute_gini_index(y_left_tree))
        return information_gain

    # Method to recursively traverse the decision tree and make prediction for a single data point
    def predict_class(self, datapoint, node):

        # Reached leaf
        if node.class_label != None: 
          return node.class_label

        # Otherwise look into the left or right subtree
        if datapoint[node.feature] <= node.threshold_value:
            return self.predict_class(datapoint, node.left_child)
        else:
            return self.predict_class(datapoint, node.right_child)
    
    # Method to start training or building the decision tree
    def learn(self, training_set):
        self.root = self.generate_decision_tree(training_set, 0)
    
    # Method to predict class for a test instance
    def classify(self, test_instance):
        return self.predict_class(test_instance, self.root)

    # Method to post-prune the decision tree
    def post_prune(self, datapoints, node):

        # If it's a leaf, nothing to do
        if node.class_label != None: 
            return 

        # For a decision-node, split the data by feature
        datapoints_to_left = np.array([datapoint for datapoint in datapoints if datapoint[node.feature] <= node.threshold_value])
        datapoints_to_right = np.array([datapoint for datapoint in datapoints if datapoint[node.feature] > node.threshold_value])

        # Prune the left and right sub-tree first
        self.post_prune(datapoints_to_left, node.left_child)
        self.post_prune(datapoints_to_right, node.right_child)

        # Prune the current node if necessary
        if(node.left_child.class_label != None and node.right_child.class_label != None):

            # Miss-classification error with pruning
            class_label_after_pruning = node.left_child.class_label if len(datapoints_to_left) > len(datapoints_to_right) else node.right_child.class_label
            miss_classification_after_pruning = (datapoints[:, -1] != class_label_after_pruning).sum() if len(datapoints) > 0 else 0
            
            # Miss-classification error without pruning
            miss_classification_before_pruning = (datapoints_to_left[:, -1] != node.left_child.class_label).sum() if len(datapoints_to_left) > 0 else 0
            miss_classification_before_pruning = miss_classification_before_pruning + ((datapoints_to_right[:, -1] != node.right_child.class_label).sum() if len(datapoints_to_right) > 0 else 0)
            
            # Prune if miss-classification error is less or equal after pruning
            if(miss_classification_after_pruning <= miss_classification_before_pruning):
              node.class_label = class_label_after_pruning
              node.left_child = None
              node.right_child = None
              return
    
    # Method that returns height of the decision tree
    def get_height(self, node):
        if node == None or node.class_label != None:
            return 0
        else:
            return max(self.get_height(node.left_child), self.get_height(node.right_child)) + 1

####################################################################################################################

# These are some of the utility/library methods which are not specific to any Decision Tree

# Utility method to compute Entropy
def compute_entropy(y):
  entropy = 0
  class_counts = np.asarray(np.unique(y, return_counts=True)).T
  for class_count in class_counts:
    class_probability = class_count[1] / len(y)
    entropy = entropy + ((-1) * class_probability * np.log2(class_probability))
  return entropy

# Utility method to compute Gini index
def compute_gini_index(y):
  gini_impurity = 0
  class_counts = np.asarray(np.unique(y, return_counts=True)).T
  for class_count in class_counts:
    class_probability = class_count[1] / len(y)
    gini_impurity = gini_impurity + (class_probability * class_probability)
  return (1 - gini_impurity)

# Utility method that returns the majority class
def find_class_label_by_majority_vote(y):
  majority_class, majority_class_count = -1, -1
  class_counts = np.asarray(np.unique(y, return_counts=True)).T
  for class_count in class_counts:
    majority_class = class_count[0] if class_count[1] > majority_class_count else majority_class
    majority_class_count = class_count[1] if class_count[1] > majority_class_count else majority_class_count
  return majority_class
    

# K-fold cross validation and accuracy calculation

#### Using 
#### K = 10
#### Impurity measurement function = entropy
#### Maximum height of the tree = Infinity(to avoid any early stopping)
#### Minimum samples to split = 2(to avoid any early stopping)

In [2]:
import math

# Method that splits the data in "K" equals portions for Cross-Validation
def KFolds_with_stratified_sampling(dataset, no_of_folds):
    np.random.seed(0)
    np.random.shuffle(dataset) 
    folds = []
    fold_size = int(len(dataset) / no_of_folds)
    class1_dataset, class2_dataset = dataset[np.where(dataset[:, -1] == 1)], dataset[np.where(dataset[:, -1] == 0)]
    class1_size_per_fold, class2_size_per_fold = int(fold_size * (len(class1_dataset) / len(dataset))), int(fold_size * (len(class2_dataset) / len(dataset)))

    # Create "K" folds maintaining same class proportions(stratified sampling)
    for i in range(no_of_folds):
        class1_data = class1_dataset[i*class1_size_per_fold : (i+1)*class1_size_per_fold]
        class2_data = class2_dataset[i*class2_size_per_fold : (i+1)*class2_size_per_fold]
        fold = np.concatenate((class1_data, class2_data), axis=0)
        np.random.seed(42)
        np.random.shuffle(fold) 
        folds.append(fold)
    return folds

# Method to train and test the decision tree using K-fold cross-validation
def run_decision_tree(no_of_folds, information_gain_strategy, minimum_datapoints_to_split, maximum_height):

    # Load data set
    wine_dataset = pd.read_csv("wine-dataset.csv")
    print(wine_dataset.describe())
    print(wine_dataset.shape)

    # Split the dataset accordingly for cross-validation
    global f
    folds = KFolds_with_stratified_sampling(np.asarray(wine_dataset), no_of_folds)
    accuracies_for_all_folds = []

    # For K-Fold cross-validation, we need to train and validate "K" times
    for i in range(no_of_folds):
        decision_tree = DecisionTree(information_gain_strategy, minimum_datapoints_to_split, maximum_height)

        # Build the training dataset from other (K-1) splits
        training_data = []
        for j in range(i):
          if(len(training_data) == 0):
            training_data = folds[j]
          else:
            training_data = np.concatenate((training_data, folds[j]), axis=0)
        for j in range(i+1, no_of_folds):
          if(len(training_data) == 0):
            training_data = folds[j]
          else:
            training_data = np.concatenate((training_data, folds[j]), axis=0)

        # Train the decision tree
        decision_tree.learn(training_data)
        f.write("Height of the tree for fold-{} : {}\n".format(i+1, decision_tree.get_height(decision_tree.root)))

        # Test the model using ith split
        results = []
        for instance in folds[i]:
            result = decision_tree.classify(instance)
            results.append( result == instance[-1])
        
        # Accuracy for each fold
        accuracy = float(results.count(True))/float(len(results))
        print("Accuracy for fold-{} is {}".format(i+1, accuracy))
        f.write("Accuracy for fold-{} is {}\n".format(i+1, accuracy))
        accuracies_for_all_folds.append(accuracy)       
    
    # Calculate average accuracy
    print("Average accuracy: %.4f" % float(sum(accuracies_for_all_folds)/len(accuracies_for_all_folds)))
    f.write("Average accuracy: %.4f\n" % float(sum(accuracies_for_all_folds)/len(accuracies_for_all_folds)))

if __name__ == "__main__":
    f.write("\n=============================10 fold cross-Validation results for the decision tree\n")
    run_decision_tree(10, "entropy", 2, math.inf)


       fixed acidity  volatile acidity  ...      alcohol      quality
count    4898.000000       4898.000000  ...  4898.000000  4898.000000
mean        6.854788          0.278241  ...    10.514267     0.216415
std         0.843868          0.100795  ...     1.230621     0.411842
min         3.800000          0.080000  ...     8.000000     0.000000
25%         6.300000          0.210000  ...     9.500000     0.000000
50%         6.800000          0.260000  ...    10.400000     0.000000
75%         7.300000          0.320000  ...    11.400000     0.000000
max        14.200000          1.100000  ...    14.200000     1.000000

[8 rows x 12 columns]
(4898, 12)
Accuracy for fold-1 is 0.8360655737704918
Accuracy for fold-2 is 0.8565573770491803
Accuracy for fold-3 is 0.8237704918032787
Accuracy for fold-4 is 0.8360655737704918
Accuracy for fold-5 is 0.8299180327868853
Accuracy for fold-6 is 0.8504098360655737
Accuracy for fold-7 is 0.8401639344262295
Accuracy for fold-8 is 0.8586065573770492


# Check accuracy using Gini index in place of Entropy

In [42]:
minimum_samples_to_split = 2
maximum_height = 23
information_gain_strategy = "gini"

# We are also applying early stoping to avoid indefinite growth of the tree
f.write("\n==========================================Results using Gini index\n")
run_decision_tree(10, information_gain_strategy, minimum_samples_to_split, maximum_height)


       fixed acidity  volatile acidity  ...      alcohol      quality
count    4898.000000       4898.000000  ...  4898.000000  4898.000000
mean        6.854788          0.278241  ...    10.514267     0.216415
std         0.843868          0.100795  ...     1.230621     0.411842
min         3.800000          0.080000  ...     8.000000     0.000000
25%         6.300000          0.210000  ...     9.500000     0.000000
50%         6.800000          0.260000  ...    10.400000     0.000000
75%         7.300000          0.320000  ...    11.400000     0.000000
max        14.200000          1.100000  ...    14.200000     1.000000

[8 rows x 12 columns]
(4898, 12)
Accuracy for fold-1 is 0.7868852459016393
Accuracy for fold-2 is 0.7848360655737705
Accuracy for fold-3 is 0.7848360655737705
Accuracy for fold-4 is 0.7889344262295082
Accuracy for fold-5 is 0.7868852459016393
Accuracy for fold-6 is 0.7848360655737705
Accuracy for fold-7 is 0.7848360655737705
Accuracy for fold-8 is 0.7827868852459017


# Pre-pruning to improve the accuracy

### We have tried to restrict the depth of the tree and minimum samples needed to split a node to avoid overfitting

In [43]:
minimum_samples_to_split = 2
maximum_height = 23
information_gain_strategy = "entropy"

f.write("\n====================================Results using Pre-pruning with entropy\n")
run_decision_tree(10, information_gain_strategy, minimum_samples_to_split, maximum_height)

       fixed acidity  volatile acidity  ...      alcohol      quality
count    4898.000000       4898.000000  ...  4898.000000  4898.000000
mean        6.854788          0.278241  ...    10.514267     0.216415
std         0.843868          0.100795  ...     1.230621     0.411842
min         3.800000          0.080000  ...     8.000000     0.000000
25%         6.300000          0.210000  ...     9.500000     0.000000
50%         6.800000          0.260000  ...    10.400000     0.000000
75%         7.300000          0.320000  ...    11.400000     0.000000
max        14.200000          1.100000  ...    14.200000     1.000000

[8 rows x 12 columns]
(4898, 12)
Accuracy for fold-1 is 0.8360655737704918
Accuracy for fold-2 is 0.8565573770491803
Accuracy for fold-3 is 0.8278688524590164
Accuracy for fold-4 is 0.8360655737704918
Accuracy for fold-5 is 0.8299180327868853
Accuracy for fold-6 is 0.8545081967213115
Accuracy for fold-7 is 0.8401639344262295
Accuracy for fold-8 is 0.8586065573770492


# Post-pruning based on miss-classification error

In [44]:

# Method to apply post-pruning with cross-validation
def run_decision_tree_with_post_pruning(no_of_folds, information_gain_strategy, minimum_datapoints_to_split, maximum_height):

    # Load data set
    wine_dataset = pd.read_csv("wine-dataset.csv")
    print(wine_dataset.describe())
    print(wine_dataset.shape)

    # Divide the complete dataset and get the K-folds
    folds = KFolds_with_stratified_sampling(np.asarray(wine_dataset), no_of_folds)

    # Here using the last fold as test set to compare the accuracy
    test_data = folds[no_of_folds - 1]

    global f
    accuracies_after_pruning = []
    accuracies_before_pruning = []

    # For K-Fold cross-validation, we need to train and validate "K" times
    for i in range(no_of_folds-1):

        pruning_data = folds[i]
        training_data = []
        for j in range(i):
          if(len(training_data) == 0):
            training_data = folds[j]
          else:
            training_data = np.concatenate((training_data, folds[j]), axis=0)
        for j in range(i+1, no_of_folds-1):
          if(len(training_data) == 0):
            training_data = folds[j]
          else:
            training_data = np.concatenate((training_data, folds[j]), axis=0)

        # Build the decision tree
        decision_tree = DecisionTree(information_gain_strategy, minimum_datapoints_to_split, maximum_height)

        # Train the decision tree
        decision_tree.learn(training_data)
        f.write("Height of the tree before pruning for fold-{} : {}\n".format(i+1, decision_tree.get_height(decision_tree.root)))

        # Accuracy before post-pruning
        results = []
        for instance in test_data:
            result = decision_tree.classify(instance)
            results.append( result == instance[-1])
        accuracy = float(results.count(True))/float(len(results))
        accuracies_before_pruning.append(accuracy)
        print("Accuracy before post-pruning is {} for fold-{}".format(accuracy, i+1))
        f.write("Accuracy before post-pruning is {} for fold-{}\n".format(accuracy, i+1))

        # Prune the decision tree
        decision_tree.post_prune(data_used_for_pruning, decision_tree.root)
        f.write("Height of the tree after pruning for fold-{} : {}\n".format(i+1, decision_tree.get_height(decision_tree.root)))
        
        # Accuracy after post-pruning
        results = []
        for instance in test_data:
            result = decision_tree.classify(instance)
            results.append( result == instance[-1])
        accuracy = float(results.count(True))/float(len(results))
        accuracies_after_pruning.append(accuracy)
        print("Accuracy after post-pruning is {} for fold-{}".format(accuracy, i+1))
        f.write("Accuracy after post-pruning is {} for fold-{}\n".format(accuracy, i+1))
        
    # Calculate average accuracy
    print("Average accuracy before post-pruning: %.4f" % float(sum(accuracies_before_pruning)/len(accuracies_before_pruning)))
    f.write("Average accuracy before post-pruning: %.4f\n" % float(sum(accuracies_before_pruning)/len(accuracies_before_pruning)))
    print("Average accuracy after post-pruning: %.4f" % float(sum(accuracies_after_pruning)/len(accuracies_after_pruning)))
    f.write("Average accuracy after post-pruning: %.4f\n" % float(sum(accuracies_after_pruning)/len(accuracies_after_pruning)))

if __name__ == "__main__":
    f.write("\n=====================================Results using Post-pruning with entropy\n")
    run_decision_tree_with_post_pruning(10, "entropy", 2, math.inf)
    f.close()

       fixed acidity  volatile acidity  ...      alcohol      quality
count    4898.000000       4898.000000  ...  4898.000000  4898.000000
mean        6.854788          0.278241  ...    10.514267     0.216415
std         0.843868          0.100795  ...     1.230621     0.411842
min         3.800000          0.080000  ...     8.000000     0.000000
25%         6.300000          0.210000  ...     9.500000     0.000000
50%         6.800000          0.260000  ...    10.400000     0.000000
75%         7.300000          0.320000  ...    11.400000     0.000000
max        14.200000          1.100000  ...    14.200000     1.000000

[8 rows x 12 columns]
(4898, 12)
Accuracy before post-pruning is 0.8381147540983607 for fold-1
Accuracy after post-pruning is 0.8442622950819673 for fold-1
Accuracy before post-pruning is 0.8545081967213115 for fold-2
Accuracy after post-pruning is 0.8463114754098361 for fold-2
Accuracy before post-pruning is 0.8442622950819673 for fold-3
Accuracy after post-pruning 